In [19]:
from bs4 import BeautifulSoup
import re
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

In [20]:
def search():
    input_search = input("inter search : ")
    location = input("inter your location 'defult is Egypt': ")
    
    search = input_search.split()
    if not location:
        location = "Egypt"
    
    if len(search) == 2:
        link = f"https://www.bayt.com/en/egypt/jobs/{search[0]}-{search[1]}-jobs/?page=1"


    elif len(search) == 3:
        link = f"hhttps://www.bayt.com/en/egypt/jobs/{search[0]}-{search[1]}-{search[2]}-jobs/?page=1"


    elif len(search) == 4:
        link = f"https://www.bayt.com/en/egypt/jobs/{search[0]}-{search[1]}-{search[2]}-{search[3]}-jobs/?page=1"

    else:
        link = f"https://www.bayt.com/en/egypt/jobs/{''.join(search)}-jobs/?page=1"


    print(location)
    return link, input_search, location

In [21]:
# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
        
    global web
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
        
    return web


In [22]:
# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
        
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 600
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(3)

In [23]:
link, input_search, location = search()
web = open_chromedriver()

inter search : data science
inter your location 'defult is Egypt': 
Egypt


C:\Users\ammar\AppData\Local\Temp\ipykernel_5624\2585799226.py:17: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


In [24]:
jobs_link = []

def get_all_jobs_link(link = link):
    web.get(link)
    page_num = 1
    
    while True:
        new_link = link[:-1] + str(page_num)
        web.get(new_link)
        
        if int(web.current_url[-1]) < page_num:
            break
        
        soup = BeautifulSoup(web.page_source, 'html.parser') # send request

        all_jobs = soup.find("ul", {"class": "media-list in-card is-spaced is-divided is-reversed has-hover-d has-rects has-free-img bb"}) 
        job_item = all_jobs.find_all("li", {"class": "has-pointer-d"})    
        links = ['https://www.bayt.com' + link.a["href"] for link in job_item]
        jobs_link.extend(links)

        scroll_down_all_the_way()
        
        print(f"- Current Page is : {page_num} .....")
        page_num = page_num + 1
    
    return jobs_link

In [25]:
links = get_all_jobs_link()

- Current Page is : 1 .....
- Current Page is : 2 .....
- Current Page is : 3 .....
- Current Page is : 4 .....
- Current Page is : 5 .....
- Current Page is : 6 .....
- Current Page is : 7 .....
- Current Page is : 8 .....
- Current Page is : 9 .....


In [26]:
len(links)

180

In [27]:
def get_all_data(links=links):
    listings = []
    
    for link in links:
        web.get(link)
        sleep(2)

        soup = BeautifulSoup(web.page_source, 'html.parser') # send request
        job_data = soup.find("div", {"class": "t-left"})

        listing = {}
        listing["job_link"] = link
        listing["label"] = input_search


        try:
            listing["job_title"] = job_data.h1.get_text(strip=True)  
            listing["job_type"] = job_data.span.get_text(strip=True)
        except:
            continue

        try:
            listing["post_time"] = job_data.find_all("li")[-1].find("span", {"class": "u-none"}).get_text(strip=True)
            listing["company_name"], listing["location"], _ = [i.get_text(strip=True) for i in job_data.find_all("li")]    

        except:
            listing["company_name"], listing["location"], listing["time"] = [i.get_text(strip=True) for i in job_data.find_all("li")]  
            
        job_desc = soup.find("div", {"class": "card-content is-spaced"})
        
        job_description = job_desc.find("div", {"class": "t-break"})
        if job_description:
            listing["job_description"] = job_description.get_text(strip=True)
        else:
            listing["job_description"] = ". ".join([i.get_text(strip=True) for i in job_desc.find_all("p")][:-2])
            
        try:
            listing["job_requirements"] = ". ".join([i.get_text(strip=True) for i in job_desc.find_all("ul")[3].find_all("li")])
        except:
            listing["job_requirements"] =  ". ".join([i.get_text(strip=True) for i in job_desc.find("div", {"class": "t-break"}).find_all("p")])           

        Skills = job_desc.find("div", {"class": "card-content is-spaced t-break print-break-before"})
        if Skills:
            listing["skills"] = ". ".join([i.get_text(strip=True) for i in Skills.find_all("li")])
        
        
        listings.append(listing)
        print("\n", "-"*100, listing, "\n")

    return listings

In [28]:
listings = get_all_data()


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/principal-data-scientist-4706595/', 'label': 'data science', 'job_title': 'Principal Data Scientist', 'job_type': 'EASY APPLY', 'post_time': '2024/02/24', 'company_name': 'Flint Consulting Ltd', 'location': 'Cairo,Egypt', 'job_description': 'Job Description:As a Principal Data Scientist, your role will be to provide strategic leadership and technical expertise in the field of data science. You will work closely with cross-functional teams, stakeholders, and executives to identify opportunities for leveraging data-driven insights and developing advanced analytics solutions. Your responsibilities will include leading complex data science projects, designing and implementing predictive models, and driving innovation in data science methodologies and techniques.Responsibilities:Lead and manage complex data science projects from inception to


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/lead-mobile-application-developer-ios-and-andriod-4895426/', 'label': 'data science', 'job_title': 'Lead Mobile Application Developer (iOS and Andriod)', 'job_type': 'EASY APPLY', 'post_time': '2024/02/25', 'company_name': 'KLD', 'location': 'Smart Village,Cairo,Egypt', 'job_description': 'KEY Responsibilities:• Design, develop, and test various mobile app use cases and features.• Select appropriate tools and approaches to achieve product goals.', 'job_requirements': '. . KEY Responsibilities:. • Design, develop, and test various mobile app use cases and features.. • Select appropriate tools and approaches to achieve product goals.. '} 


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/document-controller-4891199/', 'label': 'data scie


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/sales-executive-4884416/', 'label': 'data science', 'job_title': 'Sales Executive', 'job_type': 'EASY APPLY', 'post_time': '2024/02/18', 'company_name': 'Foodics', 'location': 'Cairo,Egypt', 'job_description': "Who Are We❓We Are Foodics! a leading restaurant management ecosystem and payment tech provider. Founded in 2014 with headquarter in Riyadh and offices across 5 countries, including UAE, Egypt, Jordan and Kuwait. We are currently serving customers and partners in over 35 different countries worldwide. Our innovative products have successfully processed over 6 billion (yes, billion with a B) orders so far! making Foodics one of the most rapidly evolving SaaS companies to ever emerge from the MENA region. Also Foodics has achieved three rounds of funding, with the latest raising $170 million in the largest SaaS funding round in MENA


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/senior-lead-quality-assurance-engineer-4886639/', 'label': 'data science', 'job_title': 'Senior/Lead Quality Assurance Engineer', 'job_type': 'Emirates Airlines', 'post_time': '2024/02/19', 'company_name': 'Emirates Airlines', 'location': 'Egypt', 'job_description': "Join the Emirates IT teamAre you keen to be part of cutting-edge projects and push the boundaries of what's possible? We’re looking for talented IT professionals to join our dynamic team of Quality Engineers in Dubai.This is your opportunity to showcase your expertise and deliver innovative projects for the world’s largest airline. Depending on your experience, you could be considered for one of two roles:Senior Quality Assurance EngineerYou’ll need at least three years’ hands-on experience developing and implementing quality engineering practices and processes. This role i


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/production-supervisor-frozen-vegetables-4868021/', 'label': 'data science', 'job_title': 'Production Supervisor– Frozen Vegetables', 'job_type': 'EASY APPLY', 'post_time': '2024/02/09', 'company_name': 'SJA', 'location': 'Egypt', 'job_description': '* Production Coordination: Coordinate and oversee the daily production activities on the line. Ensure that production targets are met, schedules are followed, and resources are effectively utilized to maximize productivity.* Team Management: Lead and supervise a team of production operators and technicians. Assign tasks, provide guidance, and conduct training to ensure a skilled and motivated workforce. Foster a positive work environment that promotes teamwork, productivity, and adherence to safety protocols.* Equipment Operation: Oversee the operation and maintenance of production line equi

...


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/senior-support-specialist-70274738/', 'label': 'data science', 'job_title': 'Senior Support Specialist', 'job_type': 'Udacity, Inc.', 'post_time': '2024/04/26', 'company_name': 'Udacity, Inc.', 'location': 'Egypt', 'job_description': 'Senior Support Specialistat Udacity EgyptUdacity is on a mission to change lives, businesses and nations throughradicaltalent transformation in digital technologies. We offer a unique and immersive online learning platform, powering corporate technical training in fields such as Artificial Intelligence, Machine Learning, Data Science, Autonomous Systems, Cloud Computing and more. Our rapidly growing global organization is revolutionizing how the enterprise market bridges the talent shortage and skills gaps during their digital transformation journey. Udacity is proud to be backed by respected investors lik


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/re-advertisement-software-and-bi-specialist-70268533/', 'label': 'data science', 'job_title': 'RE-ADVERTISEMENT-Software and BI Specialist', 'job_type': 'ICARDA', 'post_time': '2024/04/03', 'company_name': 'ICARDA', 'location': 'Egypt', 'job_description': 'Reports to:IT ManagerLocation:Cairo, EgyptMain purpose of positionThe Software and BI Specialist is responsible for full stack development and support of enterprise applications, including middleware and APIs, and database administration, Extract, Transform, and Load (ETL), reporting, and BI operations. While this role is focused on software and applications, it is a requirement that this role understands and contributes where possible to the infrastructure and operations of IT.This role may involve supervising one staff member and may necessitate national and/or international travel 


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/mlops-engineer-llm-genai-70264004/', 'label': 'data science', 'job_title': 'MLOps Engineer - LLM & GenAI', 'job_type': 'Antal International - Egypt', 'post_time': '2024/04/02', 'company_name': 'Antal International - Egypt', 'location': 'Cairo,Egypt', 'job_description': 'Degree in Information Technology, Computer Science, Artificial Intelligence or similar fields.Minimum 5 years of experience in design, development, and production-grade deployment of AI, ML, and/or\xa0NLP applications focussing on unstructured data or multi-modal data.Experience with cloud-native AI services offered by AWS (preferred) and/or Azure.Experience with LLM application development frameworks such as Langchain, LlamaIndex, Haystack, or Semantic Kernel is highly desired.Experience with good API design practices, containerization, microservices, and deployment wit


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/senior-business-analyst-70328360/', 'label': 'data science', 'job_title': 'Senior Business Analyst', 'job_type': 'Squadio', 'post_time': '2024/04/15', 'company_name': 'Squadio', 'location': 'Cairo,Egypt', 'job_description': "We are looking for a Business Analyst who will be the vital link between our information technology capacity and our business objectives by supporting and ensuring the successful completion of analytical, building, testing and deployment tasks of our software product's features.ResponsibilitiesDefine configuration specifications and business analysis requirementsPerform quality assuranceDefine reporting and alerting requirementsOwn and develop relationship with partners, working with them to optimize and enhance our integrationHelp design, document and maintain system processesReport on common sources of technical i


 ---------------------------------------------------------------------------------------------------- {'job_link': 'https://www.bayt.com/en/egypt/jobs/ad-operations-executive-70238364/', 'label': 'data science', 'job_title': 'Ad Operations Executive', 'job_type': 'Property Finder Egypt', 'post_time': '2024/04/26', 'company_name': 'Property Finder Egypt', 'location': 'Cairo,Egypt', 'job_description': "Ad Operations Executiveat Property Finder Egypt - Cairo Property Finder is the leading digital real estate platform in the Middle East and North Africa region. A UAE-born startup, Property Finder expanded its operations to Qatar, Bahrain, Saudi Arabia, and Egypt over the years. Recently, it acquired a significant stake in Zingat in Turkey. The company is one of the largest technology start-ups in the region and on a journey to becoming a Unicorn. We are aspiring to create a lighthouse technology company which will have a lasting impact in the entire tech ecosystem in our geography.Profile

In [29]:
import pandas as pd
pd.options.display.max_columns = None # donot display all rows and columns of data

df = pd.DataFrame.from_records(listings)
df

,job_link,label,job_title,job_type,post_time,company_name,location,job_description,job_requirements
0,https://www.bayt.com/en/egypt/jobs/principal-d...,data science,Principal Data Scientist,EASY APPLY,2024/02/24,Flint Consulting Ltd,"Cairo,Egypt","Job Description:As a Principal Data Scientist,...",Job Description:. As a Principal Data Scientis...
1,https://www.bayt.com/en/egypt/jobs/data-integr...,data science,Data Integration Consultant,EASY APPLY,2024/02/24,Flint Consulting Ltd,"Cairo,Egypt",Job Description:As a Data Integration Consulta...,Job Description:. As a Data Integration Consul...
2,https://www.bayt.com/en/egypt/jobs/business-su...,data science,Business Support Associate - Operational Infor...,World Food Program - Other locations,2024/02/21,World Food Program - Other locations,"Cairo,Egypt",STANDARD MINIMUM QUALIFICATIONSEducation:Unive...,Connect and share across WFP units: Facilitate...
3,https://www.bayt.com/en/egypt/jobs/lead-mobile...,data science,Lead Mobile Application Developer (iOS and And...,EASY APPLY,2024/02/25,KLD,"Smart Village,Cairo,Egypt","KEY Responsibilities:• Design, develop, and te...",". . KEY Responsibilities:. • Design, develop, ..."
4,https://www.bayt.com/en/egypt/jobs/document-co...,data science,Document Controller,Al Futtaim Group,2024/02/22,Al Futtaim Group,Egypt,Document Controller | AFGRE | CFC | Outsource ...,Document Controller | AFGRE | CFC | Outsource ...
...,...,...,...,...,...,...,...,...,...
174,https://www.bayt.com/en/egypt/jobs/staff-softw...,data science,Staff Software Engineer,ServiceNow,2024/04/22,ServiceNow,"Cairo,Egypt","Company DescriptionAt ServiceNow, our technolo...","At ServiceNow, our technology makes the world ..."
175,https://www.bayt.com/en/egypt/jobs/senior-busi...,data science,Senior Business Analyst,Squadio,2024/04/15,Squadio,"Cairo,Egypt",We are looking for a Business Analyst who will...,
176,https://www.bayt.com/en/egypt/jobs/sr-full-sta...,data science,Sr. Full-Stack Software Engineer (Nodejs),Jobzefy,2024/03/28,Jobzefy,"Cairo,Egypt",About the job Sr. Full-Stack Software Engineer...,We are looking for a Full Stack/ Backend Devel...
177,https://www.bayt.com/en/egypt/jobs/it-technica...,data science,IT Technical Support,Transmar,2024/04/26,Transmar,"Cairo,Egypt",About the job IT Technical SupportJob Summary:...,Job Summary:. Administrate and maintain the co...


In [30]:
df.isna().sum()

job_link            0
label               0
job_title           0
job_type            0
post_time           0
company_name        0
location            0
job_description     0
job_requirements    0
dtype: int64

In [31]:
def save(path, df=df, save_as_csv=True, data=listings):
    import pandas as pd
    import joblib


    joblib.dump(listings, path) # load data form PC
    
    if save_as_csv:
        df.to_csv(path, index=False)

In [32]:
file_name = "bayt_" + input_search
path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.pkl"

save(path = path)

In [33]:
df = pd.DataFrame.from_records(listings)
df.to_csv(path, index=False)

In [34]:
csv_path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.csv"

df.to_csv(csv_path, index=False)

In [35]:
listings

[{'job_link': 'https://www.bayt.com/en/egypt/jobs/principal-data-scientist-4706595/',
  'label': 'data science',
  'job_title': 'Principal Data Scientist',
  'job_type': 'EASY APPLY',
  'post_time': '2024/02/24',
  'company_name': 'Flint Consulting Ltd',
  'location': 'Cairo,Egypt',
  'job_description': 'Job Description:As a Principal Data Scientist, your role will be to provide strategic leadership and technical expertise in the field of data science. You will work closely with cross-functional teams, stakeholders, and executives to identify opportunities for leveraging data-driven insights and developing advanced analytics solutions. Your responsibilities will include leading complex data science projects, designing and implementing predictive models, and driving innovation in data science methodologies and techniques.Responsibilities:Lead and manage complex data science projects from inception to completion, ensuring timely delivery and high-quality results.Collaborate with stakehol

In [36]:
path

'F:\\NLP Apple Course\\NLP Project\\bayt_data science.pkl'